In [7]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from pyproj import CRS
import pathlib
from pathlib import Path
from shapely import wkt

import math
import codecs

import os
import re

from shapely import wkt

import folium
from folium import features
from folium import plugins

import gzip
from xml.etree.ElementTree import Element, SubElement, Comment, tostring
import xml.etree.ElementTree as ET

# import folium
from shapely.geometry import LineString, MultiLineString
import branca.colormap as cmp
from folium.plugins import Search

import time

import datetime
from datetime import timedelta

from google.cloud import storage

# set the working directory
BASE_DIR = Path.cwd()
print(BASE_DIR)

#set the project directory
project_folder = BASE_DIR.parent.parent
print(project_folder)

D:\LBNL\venv_lbl_gce\BEAM_Freight\Notebooks\GCE
D:\LBNL\venv_lbl_gce\BEAM_Freight


In [8]:
# Basic Reference Points
city_name="austin"
gcloud_bucket = "gs://beam-core-outputs"
iter_no = "0"
iter_year = "2020"
linkstats_file_path = f"{iter_no}.linkstats.csv.gz"
network_file_path = "network.csv.gz"
event_files_path = f"{iter_no}.events.csv.gz"
simulation_name = "austin_05052023"
analysis_type = "passenger"
#Check if the folder exist, if not create it.
try:
    pathlib.Path(project_folder.joinpath("Output", city_name, simulation_name, analysis_type, "plot")).mkdir(parents=True, exist_ok=False)
except:
    pass

# conversion units
meter_to_mile = 0.000621371
percent_of_samples = 0.1
mps_to_mph = 2.23694

linkstats_url = "https://storage.googleapis.com/beam-core-outputs/austin_05052023/beam/year-2020-iteration-2/ITERS/it.0/0.linkstats.csv.gz"

In [9]:
austin_rdntwrk = gpd.read_file(project_folder.joinpath("Output", city_name,simulation_name, analysis_type, "beam_network_out.geojson"))
# beam_network.plot()
austin_rdntwrk = austin_rdntwrk[["linkId","linkModes","linkLength","numberOfLanes","fromNodeId","toNodeId","linkCapacity","geometry"]]
austin_rdntwrk["linkId"] = austin_rdntwrk["linkId"].astype('string')
austintimevariantnetwork =pd.DataFrame()

linkstats = pd.read_csv(linkstats_url, compression="gzip", low_memory=True)

In [13]:
# get static maps for speed ratio
def highlight_function(feature):
    return {"fillColor": "#ffff00", "color": "#ffff00", "weight": 5,"fillOpacity": 0.40 }


color_range_pct = ["#ff0000","#ff6666","#ffb2b2","#ffdb99","#ffc966", "#ffa500",'#e5f2e5','#cce5cc','#b2d8b2','#99cc99','#7fbf7f','#66b266','#4ca64c','#329932', '#198c19','#008000']

step_pct = cmp.StepColormap(
    color_range_pct,
    vmin=0, vmax=1,
    index=[0,0.2,0.3,0.5,.6,0.7,0.80,0.85, 0.87,0.89,0.91,0.93,0.95,0.97,0.99,1.00],  #for change in the colors, not used fr linear
    caption='% Speeds Difference'    #Caption for Color scale or Legend
)


# read the road network
austin_roadnetwork = gpd.read_file(project_folder.joinpath("Output", city_name,simulation_name, analysis_type, "beam_network_out.geojson"))
austin_roadnetwork = austin_roadnetwork[["linkId","linkModes","linkLength","numberOfLanes","fromNodeId","toNodeId","linkCapacity","geometry"]]
austin_roadnetwork["linkId"] = austin_roadnetwork["linkId"].astype('string')

for time_hour in range(0,30):
    # set the map
    # pct_m = folium.Map(location=[30.266666, -97.733330], zoom_start=13, tiles="cartodbpositron")
    # get the hour and filter the linkstat file
    linkstats_time = linkstats[linkstats["hour"]==int(time_hour)].copy()
    # linkstats_time = linkstats[linkstats["hour"]==int(time_hour)].copy()
    linkstats_time=linkstats_time.add_prefix("linkstats_")
    linkstats_time.rename(columns={('linkstats_link'): 'id'}, inplace=True)
    linkstats_time["id"] = linkstats_time["id"].astype('string')
    # merge with featureclass of SF data
    comparision_network_main = austin_roadnetwork.merge(linkstats_time,left_on="linkId", right_on="id").copy()

    # linkstats
    comparision_network_main["linkstats_freespeed_mph"] = comparision_network_main["linkstats_freespeed"]*2.23694
    comparision_network_main["linkstats_congspd_mph"] = (comparision_network_main["linkstats_length"]/comparision_network_main["linkstats_traveltime"])*2.23694
    comparision_network_main["linkstats_ratio"] = comparision_network_main["linkstats_congspd_mph"] / comparision_network_main["linkstats_freespeed_mph"]

    # Condition to filter the road network, so as to reduce the load on HTML
    comparision_network = comparision_network_main.loc[(comparision_network_main["linkstats_ratio"]<0.95)].copy()

    if len(comparision_network.index)!=0:
        time_stamp = ""
        # folium
        if time_hour<30:
            time_stamp = f'{time_hour:02d}'
            layer_name = str(time_stamp)

        pct_m = folium.Map(location=[30.266666, -97.733330], zoom_start=13, tiles="cartodbpositron")

        # folium.GeoJson(comparision_network, tooltip=folium.GeoJsonTooltip(fields=["linkId"])).add_to(pct_m)

        #     layer_name=str(str(time_hour) + (' am' if time_hour < 12 else ' pm'))
        ratio_feature_group = folium.FeatureGroup(name=layer_name)

        pct_feature_group = folium.GeoJson(comparision_network,
                                       name = ("Hour - " + layer_name),
                                       style_function=lambda x: {
                                           "fillColor": step_pct(x["properties"]["linkstats_ratio"]),
                                           "color": step_pct(x["properties"]["linkstats_ratio"]),
                                           "fillOpacity": 0.8,
                                           "weight":2,
                                       },
                                       tooltip=folium.GeoJsonTooltip(fields=["linkId","linkLength", "linkstats_freespeed_mph", "linkstats_traveltime","linkstats_congspd_mph"],
                                                                     aliases=["Link ID", "Segment Length (m)", "Freespeed (mph)", "Travel time (sec)", "Congested Speed (mph)"], localize=True),
                                       popup = folium.GeoJsonPopup(fields=["linkId","linkLength", "linkstats_freespeed_mph", "linkstats_traveltime","linkstats_congspd_mph"],
                                                                     aliases=["Link ID", "Segment Length (m)", "Freespeed (mph)", "Travel time (sec)", "Congested Speed (mph)"], localize=True),
                                       highlight_function=highlight_function,
                                       zoom_on_click=True
                                      ).add_to(ratio_feature_group)
        # Add search functionality to the map
        search_link = Search(layer=pct_feature_group, geom_type="LineString", placeholders = "Search for Link ID",
                             collapsed="False", search_label = 'linkId', search_zoom = 17, position='topleft',
                             ).add_to(pct_m)
        step_pct.add_to(pct_m)
        ratio_feature_group.add_to(pct_m)
        folium.LayerControl().add_to(pct_m)

        map_title = "Ratio between Congested Speed and Free Speed"
        title_html = '''<h3 align="center" style="font-size:16px"><b>{}</b></h3>'''.format(map_title)
        pct_m.get_root().html.add_child(folium.Element(title_html))
        # save the file
        file_name = project_folder.joinpath("Output",city_name, simulation_name, analysis_type, ("linkstat_ratio_timemap_{}.html").format(time_stamp))
        pct_m.save(str(file_name))
# pct_m

In [12]:
# get static maps for vc ratio
def highlight_function(feature):
    return {"fillColor": "#ffff00", "color": "#ffff00", "weight": 5,"fillOpacity": 0.40 }

color_range_pct = ["#ff0000","#ff6666","#ffb2b2","#ffdb99","#ffc966", "#ffa500",'#e5f2e5','#cce5cc','#b2d8b2','#99cc99','#7fbf7f','#66b266','#4ca64c','#329932', '#198c19','#008000']

step_pct = cmp.StepColormap(
    list(reversed(color_range_pct)),
    vmin=0, vmax=1,
    index=[0,0.2,0.3,0.5,.6,0.7,0.80,0.85, 0.87,0.89,0.91,0.93,0.95,0.97,0.99,1.00],  #for change in the colors, not used fr linear
    caption='VC Ratio'    #Caption for Color scale or Legend
)

# read the road network
austin_roadnetwork = gpd.read_file(project_folder.joinpath("Output", city_name,simulation_name, analysis_type, "beam_network_out.geojson"))
austin_roadnetwork = austin_roadnetwork[["linkId","linkModes","linkLength","numberOfLanes","fromNodeId","toNodeId","linkCapacity","geometry"]]
austin_roadnetwork["linkId"] = austin_roadnetwork["linkId"].astype('string')

for time_hour in range(0,30):
    # set the map
    # pct_m = folium.Map(location=[30.266666, -97.733330], zoom_start=13, tiles="cartodbpositron")
    # get the hour and filter the linkstat file
    linkstats_time = linkstats[linkstats["hour"]==int(time_hour)].copy()
    # linkstats_time = linkstats[linkstats["hour"]==int(time_hour)].copy()
    linkstats_time=linkstats_time.add_prefix("linkstats_")
    linkstats_time.rename(columns={('linkstats_link'): 'id'}, inplace=True)
    linkstats_time["id"] = linkstats_time["id"].astype('string')
    # merge with featureclass of SF data
    comparision_network_main = austin_roadnetwork.merge(linkstats_time,left_on="linkId", right_on="id").copy()

    # linkstats
    comparision_network_main["linkstats_freespeed_mph"] = comparision_network_main["linkstats_freespeed"]*2.23694
    comparision_network_main["linkstats_congspd_mph"] = np.divide(comparision_network_main["linkstats_length"],comparision_network_main["linkstats_traveltime"])*2.23694
    comparision_network_main["linkstats_ratio"] = np.divide(comparision_network_main["linkstats_congspd_mph"], comparision_network_main["linkstats_freespeed_mph"])
    comparision_network_main["linkstats_vc_ratio"] = np.divide(comparision_network_main["linkstats_volume"], comparision_network_main["linkCapacity"])*(1/percent_of_samples)

    # Condition to filter the road network, so as to reduce the load on HTML
    comparision_network = comparision_network_main.loc[(comparision_network_main["linkstats_vc_ratio"]>0.20)].copy()

    if len(comparision_network.index)!=0:
        time_stamp = ""
        # folium
        if time_hour<30:
            time_stamp = f'{time_hour:02d}'
            layer_name = str(time_stamp)

        pct_m = folium.Map(location=[30.266666, -97.733330], zoom_start=13, tiles="cartodbpositron")

        # folium.GeoJson(comparision_network, tooltip=folium.GeoJsonTooltip(fields=["linkId"])).add_to(pct_m)

        #     layer_name=str(str(time_hour) + (' am' if time_hour < 12 else ' pm'))
        ratio_feature_group = folium.FeatureGroup(name=layer_name)

        pct_feature_group = folium.GeoJson(comparision_network,
                                       name = ("Hour - " + layer_name),
                                       style_function=lambda x: {
                                           "fillColor": step_pct(x["properties"]["linkstats_vc_ratio"]),
                                           "color": step_pct(x["properties"]["linkstats_vc_ratio"]),
                                           "fillOpacity": 0.8,
                                           "weight":2,
                                       },
                                       tooltip=folium.GeoJsonTooltip(fields=["linkId","linkLength", "linkstats_freespeed_mph", "linkstats_traveltime",
                                                                             "linkstats_congspd_mph", "linkCapacity", "linkstats_volume", "linkstats_vc_ratio"],
                                                                     aliases=["Link ID", "Segment Length (m)", "Freespeed (mph)", "Travel time (sec)", "Congested Speed (mph)",
                                                                              "Capacity", "Traffic Volume", "VC ratio" ], localize=True),
                                       popup = folium.GeoJsonPopup(fields=["linkId","linkLength", "linkstats_freespeed_mph", "linkstats_traveltime","linkstats_congspd_mph",
                                                                           "linkCapacity", "linkstats_volume", "linkstats_vc_ratio"],
                                                                     aliases=["Link ID", "Segment Length (m)", "Freespeed (mph)", "Travel time (sec)", "Congested Speed (mph)",
                                                                              "Capacity", "Traffic Volume", "VC ratio" ], localize=True),
                                       highlight_function=highlight_function,
                                       zoom_on_click=True
                                      ).add_to(ratio_feature_group)
        # Add search functionality to the map
        search_link = Search(layer=pct_feature_group, geom_type="LineString", placeholders = "Search for Link ID",
                             collapsed="False", search_label = 'linkId', search_zoom = 17, position='topleft',
                             ).add_to(pct_m)

        ratio_feature_group.add_to(pct_m)
        folium.LayerControl().add_to(pct_m)
        step_pct.add_to(pct_m)

        map_title = "Ratio between traffic volume and capacity"
        title_html = '''<h3 align="center" style="font-size:16px"><b>{}</b></h3>'''.format(map_title)
        pct_m.get_root().html.add_child(folium.Element(title_html))
        # save the file
        file_name = project_folder.joinpath("Output",city_name, simulation_name, analysis_type, ("linkstat_vc_ratio_timemap_{}.html").format(time_stamp))
        pct_m.save(str(file_name))
# pct_m

In [18]:
# get time-stamp maps for speed ratio
# read the road network
austin_roadnetwork = gpd.read_file(project_folder.joinpath("Output", city_name,simulation_name, analysis_type, "beam_network_out.geojson"))
austin_roadnetwork = austin_roadnetwork[["linkId","linkModes","linkLength","numberOfLanes","fromNodeId","toNodeId","linkCapacity","geometry"]]
austin_roadnetwork["linkId"] = austin_roadnetwork["linkId"].astype('string')
austin_timevariantnetwork =pd.DataFrame()

for time_hour in range(0,30):
    linkstats_time = linkstats[linkstats["hour"]==int(time_hour)].copy()
    # linkstats_time = linkstats[linkstats["hour"]==int(time_hour)].copy()
    linkstats_time=linkstats_time.add_prefix("linkstats_")
    linkstats_time.rename(columns={('linkstats_link'): 'id'}, inplace=True)
    linkstats_time["id"] = linkstats_time["id"].astype('string')

    bucket_name = "beam-core-outputs"
    destination_blob_name = f"{simulation_name}/beam/year-{iter_year}-iteration-2/ITERS/it.{iter_no}/{linkstats_file_path}"
    blob_modified_date = storage.Client().bucket(bucket_name).get_blob(destination_blob_name).updated.date()
    if int(time_hour)<24:
        date_time = blob_modified_date.strftime("%Y-%m-%d")
        time_stamp = f'{int(time_hour):02d}'
        linkstats_time["date_time"] = (date_time + " " + "{}:00:00".format(f'{int(time_hour):02d}'))
    else:
        date_time = blob_modified_date + datetime.timedelta(days=1)
        date_time = date_time.strftime("%Y-%m-%d")
        new_time = int(time_hour) - 24
        linkstats_time["date_time"] = (date_time + " " + "{}:00:00".format(f'{abs(int(new_time)):02d}'))

    # merge with featureclass of SF data
    comparision_network_main = austin_roadnetwork.merge(linkstats_time,left_on="linkId", right_on="id").copy()

    # linkstats
    comparision_network_main["linkstats_freespeed_mph"] = comparision_network_main["linkstats_freespeed"]*2.23694
    comparision_network_main["linkstats_congspd_mph"] = np.divide(comparision_network_main["linkstats_length"],comparision_network_main["linkstats_traveltime"])*2.23694
    comparision_network_main["linkstats_spd_ratio"] = np.divide(comparision_network_main["linkstats_congspd_mph"], comparision_network_main["linkstats_freespeed_mph"])
    comparision_network_main["linkstats_vc_ratio"] = np.divide(comparision_network_main["linkstats_volume"], comparision_network_main["linkCapacity"])*(1/percent_of_samples)

    comparision_network = comparision_network_main.loc[(comparision_network_main["linkstats_spd_ratio"]<0.95)].copy()

    if len(comparision_network.index)!=0:
        if int(time_hour)==0:
            austin_timevariantnetwork = comparision_network.copy()
        else:
            austin_timevariantnetwork = pd.concat([austin_timevariantnetwork,comparision_network], ignore_index=True)

# keep only selected columns fields
timevariantnetwork = austin_timevariantnetwork[["linkId", "linkModes","linkLength","numberOfLanes","linkCapacity","geometry",
                                              'linkstats_freespeed','linkstats_volume', 'linkstats_traveltime',
                                              'date_time', 'linkstats_freespeed_mph', 'linkstats_congspd_mph', 'linkstats_spd_ratio',"linkstats_vc_ratio"]]

timevariantnetwork['date_time']=pd.to_datetime(timevariantnetwork['date_time']).dt.strftime('%Y-%m-%dT%H:%M:%S')
timevariantnetwork["time"] = pd.to_datetime(timevariantnetwork["date_time"]).dt.strftime('%Y-%m-%dT%H:%M:%S')


# add more green shades for 85% --> 100%
# green_shades = ['#008000', '#198c19', '#329932', '#4ca64c', '#66b266', '#7fbf7f', '#99cc99', '#b2d8b2', '#cce5cc', '#e5f2e5']
# colors for congstd speed/freespeed ratio
color_range_pct = ["#ff0000","#ff6666","#ffb2b2","#ffdb99","#ffc966", "#ffa500",'#e5f2e5','#cce5cc','#b2d8b2','#99cc99','#7fbf7f','#66b266','#4ca64c','#329932', '#198c19','#008000']
# color_range_pct = ["#ff0000","#ff6666","#ffb2b2","#ffdb99","#ffc966", "#ffa500","#cce5cc","#99cc99","#66b266","#008000"]
step_pct = cmp.StepColormap(
    color_range_pct,
    vmin=0, vmax=1,
    index=[0,0.2,0.3,0.5,.6,0.7,0.80,0.85, 0.87,0.89,0.91,0.93,0.95,0.97,0.99,1.00],  #for change in the colors, not used fr linear
    caption='% Speeds Difference'    #Caption for Color scale or Legend
)

# colors for congstd speed/freespeed ratio
color_range_pct_vc = ['#008000',  '#329932', '#66b266', '#99cc99', '#cce5cc', '#e5f2e5', # green shade
                      '#ffa500', "#ffb732",'#ffc966', '#ffdb99', "#ffedcc", # orange shade
                     '#ffe5e5', '#ffcccc','#ffb2b2','#ff9999','#ff6666', '#ff3232', '#ff0000' ] # red shade
# color_range_pct = ["#ff0000","#ff6666","#ffb2b2","#ffdb99","#ffc966", "#ffa500","#cce5cc","#99cc99","#66b266","#008000"]
step_pct_vc = cmp.StepColormap(
    color_range_pct_vc,
    vmin=0, vmax=1,
    index=[0,0.1,0.2,0.3,0.4,0.5,
           0.55,0.6,0.65,0.7,0.75,
           0.80,0.85,0.90,0.95,0.97,0.99,1.00],  #for change in the colors, not used fr linear
    caption='Volume-to-Capacity ratio'    #Caption for Color scale or Legend
)

# colors for congested speed and freespeed
color_range = ["#ff0000","#ff6666","#ffb2b2","#ffa500","#ffc966","#ffdb99", "#cce5cc","#99cc99","#66b266","#008000"]
step = cmp.StepColormap(color_range,vmin=0, vmax=100,index=[0,5,10,15,25,35,45,55,65,100],  #for change in the colors, not used fr linear
                        caption=' Speeds (mph)'    #Caption for Color scale or Legend
                       )


def getColorMap_pct(x):
    return str(step_pct(x))

def getColorMap_pct_vc(x):
    return str(step_pct_vc(x))

def getColorMap(x):
    return str(step(x))

timevariantnetwork["fillColor_spd_ratio"] = timevariantnetwork["linkstats_spd_ratio"].apply(lambda x: getColorMap_pct(x))
timevariantnetwork["fillColor_vc_ratio"] = timevariantnetwork["linkstats_vc_ratio"].apply(lambda x: getColorMap_pct_vc(x))
timevariantnetwork["fillColor_freespeed_mph"] = timevariantnetwork["linkstats_freespeed_mph"].apply(lambda x: getColorMap(x))
timevariantnetwork["fillColor_congspd_mph"] = timevariantnetwork["linkstats_congspd_mph"].apply(lambda x: getColorMap(x))


def coords(geom):
    return list(geom.coords)

timevariantnetwork['points'] = timevariantnetwork.apply(lambda row: coords(row.geometry), axis=1)

# groupby and aggreage columns by segment_links
df1 = timevariantnetwork.groupby('linkId').agg({'linkModes':'first',
                                         'linkLength':'first',
                                         'numberOfLanes':list,
                                         'linkCapacity':list,
                                         'geometry':'first',
                                         'linkstats_freespeed':list,
                                         'linkstats_volume':list,
                                         'linkstats_traveltime':list,
                                         'date_time':list,
                                         'linkstats_freespeed_mph':list,
                                         'linkstats_congspd_mph':list,
                                         # 'linkstats_ratio':list,
                                         'linkstats_vc_ratio':list,
                                         'time':list,
                                         'fillColor_spd_ratio':list,
                                         'linkstats_volume':list,
                                         'linkstats_traveltime':list,
                                         'fillColor_freespeed_mph':list,
                                         'fillColor_congspd_mph':list,
                                         'fillColor_vc_ratio':list,
                                         'points':'first'}).reset_index()


# Create timemap for v/c ratio

def coords(geom):
    return list(geom.coords)

features_ratio = [
    {
        'type':'Feature',
        "geometry":{
            'type': 'LineString',
            'coordinates': coords(d.geometry),
        },
        'properties': {
            'times': d['time'],
            'color': "black",
            'colors':d["fillColor_spd_ratio"],
            "weight":0.6,
            "fillOpacity": 0.4,
        }
    }
    for _,d in df1.iterrows()
]

from jinja2 import Template
_template = Template("""
    {% macro script(this, kwargs) %}

        L.Control.TimeDimensionCustom = L.Control.TimeDimension.extend({
            _getDisplayDateFormat: function(date){
                var newdate = new moment(date);
                console.log(newdate)
                return newdate.format("{{this.date_options}}");
            }
        });
        {{this._parent.get_name()}}.timeDimension = L.timeDimension(
            {
                period: {{ this.period|tojson }},
            }
        );
        var timeDimensionControl = new L.Control.TimeDimensionCustom(
            {{ this.options|tojson }}
        );
        {{this._parent.get_name()}}.addControl(this.timeDimensionControl);
        var geoJsonLayer = L.geoJson({{this.data}}, {
                pointToLayer: function (feature, latLng) {
                    if (feature.properties.icon == 'marker') {
                        if(feature.properties.iconstyle){
                            return new L.Marker(latLng, {
                                icon: L.icon(feature.properties.iconstyle)});
                        }
                        //else
                        return new L.Marker(latLng);
                    }
                    if (feature.properties.icon == 'circle') {
                        if (feature.properties.iconstyle) {
                            return new L.circleMarker(latLng, feature.properties.iconstyle)
                            };
                        //else
                        return new L.circleMarker(latLng);
                    }
                    //else
                    return new L.Marker(latLng);
                },
                style: function(feature) {
                    lastIdx=feature.properties.colors.length-1
                    currIdx=feature.properties.colors.indexOf(feature.properties.color);
                    if(currIdx==lastIdx){
                        feature.properties.color = feature.properties.colors[currIdx+1]
                    }
                    else{
                        feature.properties.color =feature.properties.colors[currIdx+1]
                    }
                    return {color: feature.properties.color}
                },
                onEachFeature: function(feature, layer) {
                    if (feature.properties.popup) {
                    layer.bindPopup(feature.properties.popup);
                    }
                }
            })
        var {{this.get_name()}} = L.timeDimension.layer.geoJson(
            geoJsonLayer,
            {
                updateTimeDimension: true,
                addlastPoint: {{ this.add_last_point|tojson }},
                duration: {{ this.duration }},
            }
        ).addTo({{this._parent.get_name()}});
    {% endmacro %}
    """)


import folium
from folium.plugins import TimestampedGeoJson

m = folium.Map(location=[30.266666, -97.733330], zoom_start=13, tiles="cartodbpositron")

t=TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features_ratio,
}, transition_time=1500,loop=True,period='PT1H', add_last_point=False,auto_play=True)
t._template=_template
t.add_to(m)
step_pct.add_to(m)

# Add title
map_title = "Congested Speed to Free Speed Ratio"
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(map_title)
m.get_root().html.add_child(folium.Element(title_html))

file_name = project_folder.joinpath("Output",city_name, simulation_name, analysis_type, ("speed_ratio_timestamp_map.html"))
# file_name = BASE_DIR.parent.joinpath("exported","sf-tscore-all-trips-20PCsample-updatedRideHailFleet-updatedParking__etg",
#                                      "extended_run",("linkst_vc_ratio_timemap_iter6.html"))
m.save(str(file_name))
# m

C:\Users\Goyal\.virtualenvs\python_virtual_env-NoWs9dUT\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Goyal\.virtualenvs\python_virtual_env-NoWs9dUT\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Goyal\.virtualenvs\python_virtual_env-NoWs9dUT\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is tr

In [45]:
# get time-stamp maps for VC ratio
# read the road network
austin_roadnetwork = gpd.read_file(project_folder.joinpath("Output", city_name,simulation_name, analysis_type, "beam_network_out.geojson"))
austin_roadnetwork = austin_roadnetwork[["linkId","linkModes","linkLength","numberOfLanes","fromNodeId","toNodeId","linkCapacity","geometry"]]
austin_roadnetwork["linkId"] = austin_roadnetwork["linkId"].astype('string')
austin_timevariantnetwork =pd.DataFrame()

for time_hour in range(0,30):
    linkstats_time = linkstats[linkstats["hour"]==int(time_hour)].copy()
    # linkstats_time = linkstats[linkstats["hour"]==int(time_hour)].copy()
    linkstats_time=linkstats_time.add_prefix("linkstats_")
    linkstats_time.rename(columns={('linkstats_link'): 'id'}, inplace=True)
    linkstats_time["id"] = linkstats_time["id"].astype('string')

    bucket_name = "beam-core-outputs"
    destination_blob_name = f"{simulation_name}/beam/year-{iter_year}-iteration-2/ITERS/it.{iter_no}/{linkstats_file_path}"
    blob_modified_date = storage.Client().bucket(bucket_name).get_blob(destination_blob_name).updated.date()
    if int(time_hour)<24:
        date_time = blob_modified_date.strftime("%Y-%m-%d")
        time_stamp = f'{int(time_hour):02d}'
        linkstats_time["date_time"] = (date_time + " " + "{}:00:00".format(f'{int(time_hour):02d}'))
    else:
        date_time = blob_modified_date + datetime.timedelta(days=1)
        date_time = date_time.strftime("%Y-%m-%d")
        new_time = int(time_hour) - 24
        linkstats_time["date_time"] = (date_time + " " + "{}:00:00".format(f'{abs(int(new_time)):02d}'))

    # merge with featureclass of SF data
    comparision_network_main = austin_roadnetwork.merge(linkstats_time,left_on="linkId", right_on="id").copy()

    # linkstats
    comparision_network_main["linkstats_freespeed_mph"] = comparision_network_main["linkstats_freespeed"]*2.23694
    comparision_network_main["linkstats_congspd_mph"] = np.divide(comparision_network_main["linkstats_length"],comparision_network_main["linkstats_traveltime"])*2.23694
    comparision_network_main["linkstats_ratio"] = np.divide(comparision_network_main["linkstats_congspd_mph"], comparision_network_main["linkstats_freespeed_mph"])
    comparision_network_main["linkstats_vc_ratio"] = np.divide(comparision_network_main["linkstats_volume"], comparision_network_main["linkCapacity"])*(1/percent_of_samples)

    comparision_network = comparision_network_main.loc[(comparision_network_main["linkstats_vc_ratio"]>0.20)].copy()

    if len(comparision_network.index)!=0:
        if int(time_hour)==0:
            austin_timevariantnetwork = comparision_network.copy()
        else:
            austin_timevariantnetwork = pd.concat([austin_timevariantnetwork,comparision_network], ignore_index=True)

# keep only selected columns fields
timevariantnetwork = austin_timevariantnetwork[["linkId", "linkModes","linkLength","numberOfLanes","linkCapacity","geometry",
                                              'linkstats_freespeed','linkstats_volume', 'linkstats_traveltime',
                                              'date_time', 'linkstats_freespeed_mph', 'linkstats_congspd_mph', 'linkstats_ratio', "linkstats_vc_ratio"]]

timevariantnetwork['date_time']=pd.to_datetime(timevariantnetwork['date_time']).dt.strftime('%Y-%m-%dT%H:%M:%S')
timevariantnetwork["time"] = pd.to_datetime(timevariantnetwork["date_time"]).dt.strftime('%Y-%m-%dT%H:%M:%S')


# add more green shades for 85% --> 100%
# green_shades = ['#008000', '#198c19', '#329932', '#4ca64c', '#66b266', '#7fbf7f', '#99cc99', '#b2d8b2', '#cce5cc', '#e5f2e5']
# colors for congstd speed/freespeed ratio
color_range_pct = ["#ff0000","#ff6666","#ffb2b2","#ffdb99","#ffc966", "#ffa500",'#e5f2e5','#cce5cc','#b2d8b2','#99cc99','#7fbf7f','#66b266','#4ca64c','#329932', '#198c19','#008000']
# color_range_pct = ["#ff0000","#ff6666","#ffb2b2","#ffdb99","#ffc966", "#ffa500","#cce5cc","#99cc99","#66b266","#008000"]
step_pct = cmp.StepColormap(
    color_range_pct,
    vmin=0, vmax=1,
    index=[0,0.2,0.3,0.5,.6,0.7,0.80,0.85, 0.87,0.89,0.91,0.93,0.95,0.97,0.99,1.00],  #for change in the colors, not used fr linear
    caption='% Speeds Difference'    #Caption for Color scale or Legend
)

# colors for congstd speed/freespeed ratio
color_range_pct_vc = ['#008000',  '#329932', '#66b266', '#99cc99', '#cce5cc', '#e5f2e5', # green shade
                      '#ffa500', "#ffb732",'#ffc966', '#ffdb99', "#ffedcc", # orange shade
                     '#ffe5e5', '#ffcccc','#ffb2b2','#ff9999','#ff6666', '#ff3232', '#ff0000' ] # red shade
# color_range_pct = ["#ff0000","#ff6666","#ffb2b2","#ffdb99","#ffc966", "#ffa500","#cce5cc","#99cc99","#66b266","#008000"]
step_pct_vc = cmp.StepColormap(
    color_range_pct_vc,
    vmin=0, vmax=1,
    index=[0,0.1,0.2,0.3,0.4,0.5,
           0.55,0.6,0.65,0.7,0.75,
           0.80,0.85,0.90,0.95,0.97,0.99,1.00],  #for change in the colors, not used fr linear
    caption='Volume-to-Capacity ratio'    #Caption for Color scale or Legend
)

# colors for congested speed and freespeed
color_range = ["#ff0000","#ff6666","#ffb2b2","#ffa500","#ffc966","#ffdb99", "#cce5cc","#99cc99","#66b266","#008000"]
step = cmp.StepColormap(color_range,vmin=0, vmax=100,index=[0,5,10,15,25,35,45,55,65,100],  #for change in the colors, not used fr linear
                        caption=' Speeds (mph)'    #Caption for Color scale or Legend
                       )


def getColorMap_pct(x):
    return str(step_pct(x))

def getColorMap_pct_vc(x):
    return str(step_pct_vc(x))

def getColorMap(x):
    return str(step(x))

timevariantnetwork["fillColor_ratio"] = timevariantnetwork["linkstats_ratio"].apply(lambda x: getColorMap_pct(x))
timevariantnetwork["fillColor_vc_ratio"] = timevariantnetwork["linkstats_vc_ratio"].apply(lambda x: getColorMap_pct_vc(x))
timevariantnetwork["fillColor_freespeed_mph"] = timevariantnetwork["linkstats_freespeed_mph"].apply(lambda x: getColorMap(x))
timevariantnetwork["fillColor_congspd_mph"] = timevariantnetwork["linkstats_congspd_mph"].apply(lambda x: getColorMap(x))


def coords(geom):
    return list(geom.coords)

timevariantnetwork['points'] = timevariantnetwork.apply(lambda row: coords(row.geometry), axis=1)

# groupby and aggreage columns by segment_links
df1 = timevariantnetwork.groupby('linkId').agg({'linkModes':'first',
                                         'linkLength':'first',
                                         'numberOfLanes':list,
                                         'linkCapacity':list,
                                         'geometry':'first',
                                         'linkstats_freespeed':list,
                                         'linkstats_volume':list,
                                         'linkstats_traveltime':list,
                                         'date_time':list,
                                         'linkstats_freespeed_mph':list,
                                         'linkstats_congspd_mph':list,
                                         'linkstats_ratio':list,
                                         'linkstats_vc_ratio':list,
                                         'time':list,
                                         'fillColor_ratio':list,
                                         'linkstats_volume':list,
                                         'linkstats_traveltime':list,
                                         'fillColor_freespeed_mph':list,
                                         'fillColor_congspd_mph':list,
                                         'fillColor_vc_ratio':list,
                                         'points':'first'}).reset_index()


# Create timemap for v/c ratio

def coords(geom):
    return list(geom.coords)

features_ratio = [
    {
        'type':'Feature',
        "geometry":{
            'type': 'LineString',
            'coordinates': coords(d.geometry),
        },
        'properties': {
            'times': d['time'],
            'color': "black",
            'colors':d["fillColor_vc_ratio"],
            "weight":0.6,
            "fillOpacity": 0.4,
        }
    }
    for _,d in df1.iterrows()
]

from jinja2 import Template
_template = Template("""
    {% macro script(this, kwargs) %}

        L.Control.TimeDimensionCustom = L.Control.TimeDimension.extend({
            _getDisplayDateFormat: function(date){
                var newdate = new moment(date);
                console.log(newdate)
                return newdate.format("{{this.date_options}}");
            }
        });
        {{this._parent.get_name()}}.timeDimension = L.timeDimension(
            {
                period: {{ this.period|tojson }},
            }
        );
        var timeDimensionControl = new L.Control.TimeDimensionCustom(
            {{ this.options|tojson }}
        );
        {{this._parent.get_name()}}.addControl(this.timeDimensionControl);
        var geoJsonLayer = L.geoJson({{this.data}}, {
                pointToLayer: function (feature, latLng) {
                    if (feature.properties.icon == 'marker') {
                        if(feature.properties.iconstyle){
                            return new L.Marker(latLng, {
                                icon: L.icon(feature.properties.iconstyle)});
                        }
                        //else
                        return new L.Marker(latLng);
                    }
                    if (feature.properties.icon == 'circle') {
                        if (feature.properties.iconstyle) {
                            return new L.circleMarker(latLng, feature.properties.iconstyle)
                            };
                        //else
                        return new L.circleMarker(latLng);
                    }
                    //else
                    return new L.Marker(latLng);
                },
                style: function(feature) {
                    lastIdx=feature.properties.colors.length-1
                    currIdx=feature.properties.colors.indexOf(feature.properties.color);
                    if(currIdx==lastIdx){
                        feature.properties.color = feature.properties.colors[currIdx+1]
                    }
                    else{
                        feature.properties.color =feature.properties.colors[currIdx+1]
                    }
                    return {color: feature.properties.color}
                },
                onEachFeature: function(feature, layer) {
                    if (feature.properties.popup) {
                    layer.bindPopup(feature.properties.popup);
                    }
                }
            })
        var {{this.get_name()}} = L.timeDimension.layer.geoJson(
            geoJsonLayer,
            {
                updateTimeDimension: true,
                addlastPoint: {{ this.add_last_point|tojson }},
                duration: {{ this.duration }},
            }
        ).addTo({{this._parent.get_name()}});
    {% endmacro %}
    """)


import folium
from folium.plugins import TimestampedGeoJson

m = folium.Map(location=[30.266666, -97.733330], zoom_start=13, tiles="cartodbpositron")

t=TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features_ratio,
}, transition_time=1500,loop=True,period='PT1H', add_last_point=False,auto_play=True)
t._template=_template
t.add_to(m)
step_pct_vc.add_to(m)

# Add title
map_title = "Volume-to-Capacity Ratio"
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(map_title)
m.get_root().html.add_child(folium.Element(title_html))

file_name = project_folder.joinpath("Output",city_name, simulation_name, analysis_type, ("vc_ratio_timestamp_map.html"))
# file_name = BASE_DIR.parent.joinpath("exported","sf-tscore-all-trips-20PCsample-updatedRideHailFleet-updatedParking__etg",
#                                      "extended_run",("linkst_vc_ratio_timemap_iter6.html"))
m.save(str(file_name))
# m

C:\Users\Goyal\.virtualenvs\python_virtual_env-NoWs9dUT\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Goyal\.virtualenvs\python_virtual_env-NoWs9dUT\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Goyal\.virtualenvs\python_virtual_env-NoWs9dUT\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is tr